In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # data visualization library  
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline

In [ ]:
data = pd.read_csv('../input/data.csv')

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
# y includes our labels and x includes our features
y = data.diagnosis   # M or B 
list = ['Unnamed: 32','id','diagnosis']
x = data.drop(list,axis = 1 )  #drop two columns
x.head()

In [ ]:
ax = sns.countplot(y,label="Count") 
B, M = y.value_counts()
print('Number of Benign: ',B)
print('Number of Malignant : ',M)

In [ ]:
x.describe()

# visualization

violin plot to see similarities and check for correlation and 
swarm plot to see variance clearly for feature selection due to classification

In [ ]:
# first ten features
data_dia = y
data = x
data_n_2 = (data - data.mean()) / (data.std())              # standardization
data = pd.concat([y,data_n_2.iloc[:,0:10]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data,split=True, inner="quart")
plt.xticks(rotation=90)

In [ ]:
# Second ten features
data = pd.concat([y,data_n_2.iloc[:,10:20]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data,split=True, inner="quart")
plt.xticks(rotation=90)

In [ ]:
# Second ten features
data = pd.concat([y,data_n_2.iloc[:,20:31]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.violinplot(x="features", y="value", hue="diagnosis", data=data,split=True, inner="quart")
plt.xticks(rotation=90)

from the plots above, variable of concavity_worst and concave point_worst looks like similar but how can we decide whether they are correlated with each other or not. (Not always true but, basically if the features are correlated with each other we can drop one of them)

In [ ]:
# As an alternative of violin plot, box plot can be used
# box plots are also useful in terms of seeing outliers
# I do not visualize all features with box plot
# If you want, you can visualize other features as well.
plt.figure(figsize=(10,10))
sns.boxplot(x="features", y="value", hue="diagnosis", data=data)
plt.xticks(rotation=90)

to compare two features deeper, we can use joint plot.it correlate.
0.86 is enough to say they correlate.
Note: we are not choosing feature, we are just looking to have an idea about them.

In [ ]:
sns.jointplot(x.loc[:,'concavity_worst'], x.loc[:,'concave points_worst'], kind="regg", color="#ce1414")

What about three or more feauture comparision ? we can use pair grid plot. Notice that radius_worst, perimeter_worst and area_worst are correlated as it can be seen pair grid plot.
We'll use these discoveries for feature selection.

In [ ]:
sns.set(style="white")
df = x.loc[:,['radius_worst','perimeter_worst','area_worst']]
g = sns.PairGrid(df, diag_sharey=False)
g.map_lower(sns.kdeplot, cmap="Blues_d")
g.map_upper(plt.scatter)
g.map_diag(sns.kdeplot, lw=3)

# let's try swarm plot

In [ ]:
sns.set(style="whitegrid", palette="muted")
data_dia = y
data = x
data_n_2 = (data - data.mean()) / (data.std())              # standardization
data = pd.concat([y,data_n_2.iloc[:,0:10]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
tic = time.time()
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)

plt.xticks(rotation=90)

In [ ]:
data = pd.concat([y,data_n_2.iloc[:,10:20]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)
plt.xticks(rotation=90)

In [ ]:
data = pd.concat([y,data_n_2.iloc[:,20:31]],axis=1)
data = pd.melt(data,id_vars="diagnosis",
                    var_name="features",
                    value_name='value')
plt.figure(figsize=(10,10))
sns.swarmplot(x="features", y="value", hue="diagnosis", data=data)
toc = time.time()
plt.xticks(rotation=90)
print("swarm plot time: ", toc-tic ," s")

Here,you can see variance clearly.  This help us to know features that are good for classification. above, area_worst seperates malignant and benign to a reasonable extent. Hovewer, smoothness_se are mixed so it is not the best to be used for classification.

# Heatmap to observe all correlation between features

In [ ]:
#correlation map
f,ax = plt.subplots(figsize=(18, 18))
sns.heatmap(x.corr(), annot=True, linewidths=.5, fmt= '.1f',ax=ax)

observation

The radius, parameter and area are highly correlated as expected from their relation so from these we will use anyone of them

Compactness_mean, concavity_mean and concavepoint_mean are highly correlated so we will use compactness_mean from here

So selected Parameter for use is perimeter_mean, texture_mean, compactness_mean, symmetry_mean*

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split

# split data train 70 % and test 30 %
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

clf = LogisticRegression(random_state = 2)
clf.fit(x_train, y_train)

In [ ]:
y_pred = clf.predict(x_test)

# Accuracy

Accuracy is the most intuitive performance Metric. It's the number of correct predictions over all predictions made. It's the default performance metric in scikit Learn. We can compute the accuracy in two ways: using the accuracy_score() method or the score() method:



In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_true= y_test, y_pred= y_pred)

In [ ]:
clf.score(x_test, y_test)

# Confusion Matrix

A much better way to evaluate the performance is to look at the confusion matrix. The confusion matrix give us information about the correct predictions and the errors. It's a two-by-two array, where the rows correspond to the true classes and the columns correspond to the predicted classes.

Let's look at the code to output confusion matrix for our prediction task and explain what TP, TN, FP, FN means

In [ ]:
from sklearn.metrics import f1_score,confusion_matrix

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
cm = confusion_matrix(y_test,clf.predict(x_test))
sns.heatmap(cm,annot=True,fmt="d")

# Random Forest & Confusion Matrix

In [ ]:
#from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# split data train 70 % and test 30 %
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

#random forest classifier with n_estimators=10 (default)
clf_rf = RandomForestClassifier(random_state=43)      
clr_rf = clf_rf.fit(x_train,y_train)

ac = accuracy_score(y_test,clf_rf.predict(x_test))
print('Accuracy is: ',ac)
cm = confusion_matrix(y_test,clf_rf.predict(x_test))
sns.heatmap(cm,annot=True,fmt="d")

Accuracy is almost 95% and as it can be seen in confusion matrix, we make few wrong prediction. Now lets see other feature selection methods to find better results.

# Univariate feature selection and random forest classification

In univariate feature selection, we will use SelectKBest that removes all but the k highest scoring features. http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest

In this method we need to choose how many features we will use. For example, will k (number of features) be 5 or 10 or 15? let'schoose k = 5 and find best 5 features.

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
# find best scored 5 features
select_feature = SelectKBest(chi2, k=5).fit(x_train, y_train)

In [ ]:
print('Score list:', select_feature.scores_)
print('Feature list:', x_train.columns)

Best 5 feature to classify is that area_mean, area_se, texture_mean, concavity_worst and concavity_mean. So lets se what happens if we use only these best scored 5 feature.

In [ ]:
x_train_2 = select_feature.transform(x_train)
x_test_2 = select_feature.transform(x_test)
#random forest classifier with n_estimators=10 (default)
clf_rf_2 = RandomForestClassifier()      
clr_rf_2 = clf_rf_2.fit(x_train_2,y_train)
ac_2 = accuracy_score(y_test,clf_rf_2.predict(x_test_2))
print('Accuracy is: ',ac_2)
cm_2 = confusion_matrix(y_test,clf_rf_2.predict(x_test_2))
sns.heatmap(cm_2,annot=True,fmt="d")

# We are good to go with these!!!

The rest should be figures and business impact

I will forward my findings as soon as possible.

Thanks